![title](image.png)

<h1>Feather Vision</h1>


<style>
	p {
		line-height: 180%;
	}
</style>
<h2>About the data</h2>
<p>
We have decided to use birds classification dataset from <a href="https://www.kaggle.com/datasets/gpiosenka/100-bird-species">kaggle</a>. It has a large number of training images (almost 85,000!) and includes over 500 different kinds of birds. This will help our model learn to identify lots of different bird species. We wanted a dataset with lots of high-quality images, good bird visibility, and this one fits us well. Important note is that all the images are photographed and not AI generated. While there are a few things to keep in mind, like each picture only having one bird, which can be good or bad, and also the images are 80% male, we believe this dataset is a good choice for our bird classification project.
</p>

<h3>Size and type of data</h3>
<p>
As mentioned earlier, the dataset contains almost 85K high quality images.   
<ul>
	<li>The images are sized 224 X 224 and are in JPG format.</li>
	<li>The average image size is around 20kB.</li>
	<li>The images are photographed during all seasons and it should contain species from every continent.</li>
</ul>
</p>